In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
from scipy.linalg import orth

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')
sns.set_color_codes()

In [5]:
from ridge import *

In [6]:
def f(x):
    if len(x.shape) == 1:
        return 1000*x[0]**2 + x[1]**2 + x[2]**2
    elif len(x.shape) == 2:
        return 1000*x[:,0]**2 + x[:,1]**2 + x[:,2]**2

In [10]:
N = 1000
pra = PolynomialRidgeApproximation(subspace_dimension = 2, degree = 2, gtol = 0, rtol = 0, ftol = 0, xtol = 1e-10, n_init = 10)
U_fit = []
for i in range(10000):
    np.random.seed(i)
    X = np.random.uniform(-1,1, size = (N,10))
    fX = f(X)
    pra.fit(X, fX)
    U_fit.append(pra.U)

In [ ]:
angles = np.zeros((len(U_fit),3))
for i, U in enumerate(U_fit):
    for j in range(3):
        x = np.zeros((10,1))
        x[j] =  1
        angles[i,j] = np.arccos(np.linalg.svd(np.dot(x.T, U), compute_uv = False))*180/np.pi

In [ ]:
fig, axes = plt.subplots(1,3, figsize = (9,3))
from sklearn.neighbors import KernelDensity
from pgf import PGF

for i, ax in enumerate(axes):
    if i == 0:
        kde = KernelDensity(bandwidth = 5e-4)
        x_grid = np.linspace(0,0.02, 500)
    else:
        kde = KernelDensity(bandwidth = 1)
        x_grid = np.linspace(0,90, 500)
    kde.fit(angles[:,i].reshape(-1,1))
    
    y = np.exp(kde.score_samples(x_grid.reshape(-1,1)))
    ax.plot(x_grid, y)
    pgf = PGF()
    pgf.add('x', x_grid)
    pgf.add('den', y)
    pgf.write('fig_consistency_%d.dat' % i)
fig.tight_layout()